In [4]:
# yo chai ho hai
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
pip install git+https://github.com/fbcotter/pytorch_wavelets#egg=pytorch_wavelets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/fbcotter/pytorch_wavelets to /tmp/pip-install-uajb8e2l/pytorch-wavelets_aa81b8f012934f0ab785622a400e83e2
  Running command git clone --filter=blob:none --quiet https://github.com/fbcotter/pytorch_wavelets /tmp/pip-install-uajb8e2l/pytorch-wavelets_aa81b8f012934f0ab785622a400e83e2
  Resolved https://github.com/fbcotter/pytorch_wavelets to commit 9a0c507f04f43c5397e384bb6be8340169b2fd9a
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch_wavelets: filename=pytorch_wavelets-1.3.0-py3-none-any.whl size=54869 sha256=cfe7e3c08ad8ce26255b534ae18af7ce15c58d0bc76f3eac15c57d72aee766d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-ec3yq77b/wheels/0c/15/77/b39e24755082220b25f2d5b1bae3aa04bc4e1fc60056885daf
Successfully built pytorch_wavelets


In [6]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np 
import pandas as pd
import torchvision
import matplotlib.pyplot as plt
from PIL import Image
import joblib
from google.colab import drive
from pytorch_wavelets import DWTForward, DWTInverse # (or import DWT, IDWT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# img = Image.new("RGB", (512, 512), (0, 0, 0))

# # Save the image as a PNG file
# img.save("/content/drive/MyDrive/Colab Notebooks/Steganography/Black.png")
# img = Image.new("RGB", (512, 512), (255, 255, 255))

# # Save the image as a PNG file
# img.save("/content/drive/MyDrive/Colab Notebooks/Steganography/White.png")

In [8]:
# img = Image.open('/content/drive/MyDrive/Colab Notebooks/Steganography/Invert.png')

# # Keep only the first 3 channels (red, green, and blue)
# img = img.convert('RGB')

# # Save the converted 3-channel image
# img.save('/content/drive/MyDrive/Colab Notebooks/Steganography/Invert.png')
# img = Image.open('/content/drive/MyDrive/Colab Notebooks/Steganography/image.png')

# # Keep only the first 3 channels (red, green, and blue)
# img = img.convert('RGB')

# # Save the converted 3-channel image
# img.save('/content/drive/MyDrive/Colab Notebooks/Steganography/image.png')


In [9]:
root_dir = '/content/drive/MyDrive/Colab Notebooks/'
dataset_path = os.path.join(root_dir,'Dataset256')

train_csv = os.path.join(root_dir,'Dataset_x256.csv')
test_csv = os.path.join(root_dir,'Steganography/validation_dataset.csv')
train_folder = os.path.join(dataset_path,'train')
validation_folder = os.path.join(dataset_path,'Valid')

In [10]:
cover = '/content/drive/MyDrive/Colab Notebooks/Steganography/NikeshS.jpg'
secret = '/content/drive/MyDrive/Colab Notebooks/Steganography/NikeshC.jpg'

dataset = [{
        'cover_image':cover,
        'secret_image':secret
    }]
dataframe = pd.DataFrame(dataset)
cover = Image.open(cover)
secret = Image.open(secret)


In [11]:
%matplotlib inline

In [12]:

IMG_SIZE = 256
LEARNING_RATE  = 0.001
COVER_LOSS_WEIGHT = 1
SECRET_LOSS_WEIGHT = 1
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 1
EPOCHS = 1000
DECODER_LOSS_WEIGHT = 1

In [13]:
imagetransformation = {
    'train_transforms':torchvision.transforms.Compose([torchvision.transforms.ToTensor()]),
    'valid_transforms':torchvision.transforms.Compose([torchvision.transforms.ToTensor()]),
    # 'test_transforms':torchvision.transforms.Compose([torchvision.transforms.Resize(cover.size),torchvision.transforms.ToTensor()])
}

In [14]:
#conversion of image into resized format and return of trnasformed image
class SteganoDataset(torch.utils.data.Dataset):
    def __init__(self,dataset_csv,transforms,type_of_dataset,size='complete'):
        self.dataset = pd.read_csv(dataset_csv)
        # self.dataset = dataset_csv
        self.dataset = self.dataset.reset_index(drop=True)
        if size !='complete':
            self.dataset = self.dataset[:2]
        self.transforms = transforms
        self.type = type_of_dataset
    
    def __getitem__(self,index):
        cover_image = self.dataset.iloc[index]['cover_image']
        secret_image = self.dataset.iloc[index]['secret_image']
        cover_image = Image.open(os.path.join(dataset_path,'cover',cover_image))
        secret_image = Image.open(os.path.join(dataset_path,'secret',secret_image))
        transformed_cover_image = self.transforms(cover_image)
        transformed_secret_image = self.transforms(secret_image)
        return {
                'cover_image':transformed_cover_image,
                'secret_image':transformed_secret_image,
            }
        
    
    def __len__(self):
      return len(self.dataset)

In [15]:
class PrepNetwork(nn.Module):
    def __init__(self,xfm):
        super().__init__()
        self.xfm = xfm
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=(4,4),stride=2,padding=1)
        self.bn1 = nn.BatchNorm2d(6)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=(4,4),stride=2,padding=1)
        self.bn2 = nn.BatchNorm2d(12)

        # Define activation functions
        self.relu = nn.ReLU()

    def forward(self,secret_image):
        secret_cA, [secret_cD] = self.xfm(secret_image)
        secret_A = self.relu(self.bn1(self.conv1(secret_cA)))
        secret_cH = self.relu(self.bn1(self.conv1(secret_cD[:,:,0,:,:])))
        secret_cV = self.relu(self.bn1(self.conv1(secret_cD[:,:,1,:,:])))
        secret_cD = self.relu(self.bn1(self.conv1(secret_cD[:,:,2,:,:])))

        secret_H = self.relu(self.bn2(self.conv2(secret_cH)))
        secret_V = self.relu(self.bn2(self.conv2(secret_cV)))
        secret_D = self.relu(self.bn2(self.conv2(secret_cD)))

        secret_CD = torch.cat([secret_H,secret_V,secret_D],dim=1)

        return secret_A, secret_CD

class HidingNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=(4,4),stride=2,padding=1)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=(4,4),stride=2,padding=1)
        self.bn12 = nn.BatchNorm2d(12)
        self.conv3 = nn.Conv2d(in_channels=18,out_channels=36,kernel_size=(4,4),stride=2,padding=1)
        self.bn36 = nn.BatchNorm2d(36)
        self.conv4 = nn.Conv2d(in_channels=72,out_channels=144,kernel_size=(4,4),stride=2,padding=1)
        self.bn144 = nn.BatchNorm2d(144)
        self.conv5 = nn.Conv2d(in_channels=144,out_channels=288,kernel_size=(4,4),stride=2,padding=1)
        self.bn288 = nn.BatchNorm2d(288)
        self.conv6 = nn.Conv2d(in_channels=288,out_channels=576,kernel_size=(4,4),stride=2,padding=1)
        self.bn576 = nn.BatchNorm2d(576)
        self.conv7 = nn.Conv2d(in_channels=576,out_channels=1152,kernel_size=(4,4),stride=2,padding=1)
        
        self.deconv7 = nn.ConvTranspose2d(in_channels=1152,out_channels=576,kernel_size=(4,4),stride=2,padding=1)
        self.deconv6 = nn.ConvTranspose2d(in_channels=1152,out_channels=288,kernel_size=(4,4),stride=2,padding=1)
        self.deconv5 = nn.ConvTranspose2d(in_channels=576,out_channels=144,kernel_size=(4,4),stride=2,padding=1)
        self.deconv4 = nn.ConvTranspose2d(in_channels=288,out_channels=72,kernel_size=(4,4),stride=2,padding=1)
        self.bn72 = nn.BatchNorm2d(72)
        self.deconv3 = nn.ConvTranspose2d(in_channels=108,out_channels=54,kernel_size=(4,4),stride=2,padding=1)
        self.bn54 = nn.BatchNorm2d(54)
        self.deconv2 = nn.ConvTranspose2d(in_channels=66,out_channels=33,kernel_size=(4,4),stride=2,padding=1)
        self.bn33 = nn.BatchNorm2d(33)
        self.deconv1 = nn.ConvTranspose2d(in_channels=39,out_channels=3,kernel_size=(4,4),stride=2,padding=1)

        # Define activation functions
        self.relu = nn.ReLU()
        self.lrelu = nn.LeakyReLU(0.2)
        self.sigmoid = nn.Sigmoid()        

    def forward(self,secret_A, secret_CD,cover_image):
        output_1 = self.relu(self.conv1(cover_image))
        output_2 = self.relu(self.bn12(self.conv2(output_1)))
        concat_output_2 = torch.cat([secret_A,output_2],dim=1)        
        output_3 = self.relu(self.bn36(self.conv3(concat_output_2)))
        concat_output_3 = torch.cat([secret_CD,output_3],dim=1) 
        output_4 = self.relu(self.bn144(self.conv4(concat_output_3)))
        output_5 = self.relu(self.bn288(self.conv5(output_4)))
        output_6 = self.relu(self.bn576(self.conv6(output_5)))
        output_7 = self.relu(self.conv7(output_6))

        hidden_7 = self.lrelu(self.bn576(self.deconv7(output_7)))
        concat_hidden_6 = torch.cat([output_6,hidden_7],dim=1) 
        hidden_6 = self.lrelu(self.bn288(self.deconv6(concat_hidden_6)))
        concat_hidden_5 = torch.cat([output_5,hidden_6],dim=1) 
        hidden_5 = self.lrelu(self.bn144(self.deconv5(concat_hidden_5)))
        concat_hidden_4 = torch.cat([output_4,hidden_5],dim=1)
        hidden_4 = self.lrelu(self.bn72(self.deconv4(concat_hidden_4)))
        concat_hidden_3 = torch.cat([output_3,hidden_4],dim=1) 
        hidden_3 = self.lrelu(self.bn54(self.deconv3(concat_hidden_3)))
        concat_hidden_2 = torch.cat([output_2,hidden_3],dim=1) 
        hidden_2 = self.lrelu(self.bn33(self.deconv2(concat_hidden_2)))
        concat_hidden_1 = torch.cat([output_1,hidden_2],dim=1) 
        hidden_image = self.sigmoid(self.deconv1(concat_hidden_1))
      
        return hidden_image

In [16]:
class Encoder(nn.Module):
    def __init__(self,prep_network,hiding_network):
        super().__init__()
        self.prep_network = prep_network
        self.hiding_network = hiding_network
    
    def forward(self,cover_image,secret_image):
        secret_A, secret_CD = self.prep_network(secret_image)
        
        hidden_image = self.hiding_network(secret_A, secret_CD,cover_image)
#         hidden_image = (0.01**0.5)*torch.randn(hidden_image.size(),device=device)
        return hidden_image

In [17]:
# class RevealNetwork(nn.Module):
#     def __init__(self,ifm):
#         super().__init__()
#         self.ifm = ifm
#         self.conv1 = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=(5,5),stride=2,padding=2)
#         self.bn64 = nn.BatchNorm2d(64)
#         self.conv2 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=(5,5),stride=2,padding=2)
#         self.bn128 = nn.BatchNorm2d(128)
#         self.conv3 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=(5,5),stride=2,padding=2)
#         self.bn256 = nn.BatchNorm2d(256)
#         self.deconv1 = nn.ConvTranspose2d(in_channels=256,out_channels=128,kernel_size=(5,5),stride=2,padding=2)
#         self.deconv2 = nn.ConvTranspose2d(in_channels=128,out_channels=64,kernel_size=(5,5),stride=2,padding=2)

#         self.deconvC1 = nn.ConvTranspose2d(in_channels=64,out_channels=9,kernel_size=(3,3),stride=1,padding=1)
#         self.bn9 = nn.BatchNorm2d(9)
#         self.deconvC2 = nn.ConvTranspose2d(in_channels=9,out_channels=3,kernel_size=(3,3),stride=1,padding=1)   

#         # Define activation functions
#         self.relu = nn.ReLU()
#         self.lrelu = nn.LeakyReLU(0.2)

#     def forward(self,hidden_image):
        
#         output_1 = self.relu(self.bn64(self.conv1(hidden_image)))
#         output_2 = self.relu(self.bn128(self.conv2(output_1)))
#         output_3 = self.relu(self.bn256(self.conv3(output_2)))
        
#         output_4 = self.lrelu(self.bn128(self.deconv1(output_3)))
#         output_5 = self.lrelu(self.bn64(self.deconv2(output_4)))

#         output_A = self.lrelu(self.bn9(self.deconvC1(output_5)))
#         revealed_A = self.lrelu(self.deconvC2(output_A))

#         output_H = self.lrelu(self.bn9(self.deconvC1(output_5)))
#         revealed_H = self.lrelu(self.deconvC2(output_H))

#         output_V = self.lrelu(self.bn9(self.deconvC1(output_5)))
#         revealed_V = self.lrelu(self.deconvC2(output_V))

#         output_D = self.lrelu(self.bn9(self.deconvC1(output_5)))
#         revealed_D = self.lrelu(self.deconvC2(output_D))

#         revealed_C = torch.stack([revealed_H,revealed_V,revealed_D], dim=2)
#         revealed_image = self.ifm((revealed_A,[revealed_C]))
#         # if coeff:
#         #     return (revealed_A,[revealed_C])
#         # else:
#         return revealed_image

In [110]:
class RevealNetwork(nn.Module):
    def __init__(self,ifm):
        super().__init__()
        self.ifm = ifm
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=39,kernel_size=(5,5),stride=2,padding=2)
        self.bn39 = nn.BatchNorm2d(39)
        self.conv2 = nn.Conv2d(in_channels=39,out_channels=66,kernel_size=(5,5),stride=2,padding=2)
        self.bn66 = nn.BatchNorm2d(66)
        self.deconv0 = nn.ConvTranspose2d(in_channels=66,out_channels=18,kernel_size=(5,5),stride=1,padding=2)
        self.bn18 = nn.BatchNorm2d(18)
        self.conv3 = nn.Conv2d(in_channels=66,out_channels=108,kernel_size=(5,5),stride=2,padding=2)
        self.bn108 = nn.BatchNorm2d(108)

        self.deconv1 = nn.ConvTranspose2d(in_channels=108,out_channels=72,kernel_size=(5,5),stride=1,padding=2)
        self.bn72 = nn.BatchNorm2d(72)
        self.deconv2 = nn.ConvTranspose2d(in_channels=72,out_channels=36,kernel_size=(5,5),stride=1,padding=2)
        self.bn36 = nn.BatchNorm2d(36)

        self.deconv3 = nn.ConvTranspose2d(in_channels=36,out_channels=12,kernel_size=(5,5),stride=1,padding=2)
        self.bn12 = nn.BatchNorm2d(12)
        self.deconv4 = nn.ConvTranspose2d(in_channels=18,out_channels=6,kernel_size=(5,5),stride=1,padding=2)
        self.deconv5 = nn.ConvTranspose2d(in_channels=12,out_channels=6,kernel_size=(3,3),stride=2,padding=1)
        self.deconv6 = nn.ConvTranspose2d(in_channels=6,out_channels=3,kernel_size=(3,3),stride=2,padding=1)

        # Define activation functions
        self.relu = nn.ReLU()
        self.lrelu = nn.LeakyReLU(0.2)

    def forward(self,hidden_image):
        print(hidden_image.size())
        output_1 = self.relu(self.bn39(self.conv1(hidden_image)))
        print(output_1.size())
        output_2 = self.relu(self.bn66(self.conv2(output_1)))
        print(output_2.size())
        output_3 = self.relu(self.bn108(self.conv3(output_2)))
        print(output_3.size())
        output_21 = self.lrelu(self.bn18(self.deconv0(output_2)))
        print(output_21.size())
        output_C3 = self.lrelu(self.bn72(self.deconv1(output_3)))
        print(output_21.size())
        output_C2 = self.lrelu(self.bn36(self.deconv2(output_C3)))

        output_A0 = self.lrelu(self.deconv4(output_21))
        output_A = self.lrelu(self.deconv6(output_A0))

        output_CH1 = self.lrelu(self.bn12(self.deconv3(output_C2)))
        output_CH0 = self.lrelu(self.deconv5(output_CH1))
        output_H = self.lrelu(self.deconv6(output_CH0))
        output_CV1 = self.lrelu(self.bn12(self.deconv3(output_C2)))
        output_CV0 = self.lrelu(self.deconv5(output_CV1))
        output_V = self.lrelu(self.deconv6(output_CV0))
        output_CD1 = self.lrelu(self.bn12(self.deconv3(output_C2)))
        output_CD0 = self.lrelu(self.deconv5(output_CD1))
        output_D = self.lrelu(self.deconv6(output_CD0))
        output_C = torch.stack([output_H,output_V,output_D], dim=2)
        print(output_C.size())
        print(output_A.size())
        revealed_image = self.ifm((output_A,[output_C]))
        # if coeff:
        #     return (revealed_A,[revealed_C])
        # else:
        return revealed_image

In [111]:
# class PrepNetwork(nn.Module):
#     def __init__(self,xfm):
#         super().__init__()
#         self.xfm = xfm
#         self.conv1 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=(4,4),stride=2,padding=1)
#         self.bn1 = nn.BatchNorm2d(6)
#         self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=(4,4),stride=2,padding=1)
#         self.bn2 = nn.BatchNorm2d(12)

#         # Define activation functions
#         self.relu = nn.ReLU()

#     def forward(self,secret_image):
#         secret_cA, [secret_cD] = self.xfm(secret_image)
#         secret_A = self.relu(self.bn1(self.conv1(secret_cA)))
#         secret_cH = self.relu(self.bn1(self.conv1(secret_cD[:,:,0,:,:])))
#         secret_cV = self.relu(self.bn1(self.conv1(secret_cD[:,:,1,:,:])))
#         secret_cD = self.relu(self.bn1(self.conv1(secret_cD[:,:,2,:,:])))

#         secret_H = self.relu(self.bn2(self.conv2(secret_cH)))
#         secret_V = self.relu(self.bn2(self.conv2(secret_cV)))
#         secret_D = self.relu(self.bn2(self.conv2(secret_cD)))

#         secret_CD = torch.cat([secret_H,secret_V,secret_D],dim=1)

#         return secret_A, secret_CD

# class HidingNetwork(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.deconv1 = nn.ConvTranspose2d(out_channels=3,in_channels=6,kernel_size=(4,4),stride=2,padding=1)
#         self.deconv2 = nn.ConvTranspose2d(out_channels=6,in_channels=12,kernel_size=(4,4),stride=2,padding=1)
#         self.bn18 = nn.BatchNorm2d(18)
#         self.deconv3 = nn.ConvTranspose2d(out_channels=18,in_channels=36,kernel_size=(4,4),stride=2,padding=1)
#         self.bn72 = nn.BatchNorm2d(72)
#         self.deconv4 = nn.ConvTranspose2d(out_channels=72,in_channels=144,kernel_size=(4,4),stride=2,padding=1)
#         self.bn144 = nn.BatchNorm2d(144)
#         self.deconv5 = nn.ConvTranspose2d(out_channels=144,in_channels=288,kernel_size=(4,4),stride=2,padding=1)
#         self.bn288 = nn.BatchNorm2d(288)
#         self.deconv6 = nn.ConvTranspose2d(out_channels=288,in_channels=576,kernel_size=(4,4),stride=2,padding=1)
#         self.bn576 = nn.BatchNorm2d(576)
#         self.deconv7 = nn.ConvTranspose2d(out_channels=576,in_channels=1152,kernel_size=(4,4),stride=2,padding=1)
        
#         self.conv7 = nn.Conv2d(out_channels=1152,in_channels=576,kernel_size=(4,4),stride=2,padding=1)
#         self.conv6 = nn.Conv2d(out_channels=1152,in_channels=288,kernel_size=(4,4),stride=2,padding=1)
#         self.conv5 = nn.Conv2d(out_channels=576,in_channels=144,kernel_size=(4,4),stride=2,padding=1)
#         self.conv4 = nn.Conv2d(out_channels=288,in_channels=72,kernel_size=(4,4),stride=2,padding=1)
#         self.bn108 = nn.BatchNorm2d(108)
#         self.conv3 = nn.Conv2d(out_channels=108,in_channels=54,kernel_size=(4,4),stride=2,padding=1)
#         self.bn66 = nn.BatchNorm2d(66)
#         self.conv2 = nn.Conv2d(out_channels=66,in_channels=33,kernel_size=(4,4),stride=2,padding=1)
#         self.bn39 = nn.BatchNorm2d(39)
#         self.conv1 = nn.Conv2d(out_channels=39,in_channels=3,kernel_size=(4,4),stride=2,padding=1)

#         # Define activation functions
#         self.relu = nn.ReLU()
#         self.lrelu = nn.LeakyReLU(0.2)
#         self.sigmoid = nn.Sigmoid()        

#     def forward(self,secret_A, secret_CD,cover_image):
#         output_1 = self.relu(self.conv1(cover_image)) #39
#         output_12 = self.relu(self.deconv1(output_1)) #33
#         # output11, output12 = torch.split(output_1, [6, 33], dim=1) 
#         output_2 = self.relu(self.bn66(self.conv2(output12)))
#         output_2 = self.relu(self.bn66(self.deconv2(output12)))
#         output21, output22 = torch.split(output_2, [12, 54], dim=1) 
               
#         output_3 = self.relu(self.bn108(self.conv3(output22)))
#         output31, output32 = torch.split(output_3, [36, 72], dim=1)
#         output_4 = self.relu(self.bn288(self.conv4(output32)))
#         output41, output42 = torch.split(output_4, [144, 144], dim=1)

#         secret_2 = self.lrelu(self.bn72(self.deconv7(output41)))
#         secret21, secret22 = torch.split(secret_2, [36, 36], dim=1) 
#         secretH2, secretV2, secretD2 = torch.split(secret21, [12, 12, 12], dim=1) 
#         secret_1 = self.lrelu(self.bn18(self.deconv6(secret22)))
#         secret11, secret12 = torch.split(secret_2, [6, 12], dim=1) 
#         secret_A1 = self.lrelu(self.bn3(self.deconv5(secret11)))
#         secret_H1 = self.lrelu(self.bn144(self.deconv5(secretH2)))
#         secret_V1 = self.lrelu(self.bn144(self.deconv5(secretV2)))
#         secret_D1 = self.lrelu(self.bn144(self.deconv5(secretD2)))


#         secret_A = self.lrelu(self.bn72(self.deconv4(secret_A1)))
#         secret_H = self.lrelu(self.bn72(self.deconv4(secret_H1)))
#         secret_V = self.lrelu(self.bn72(self.deconv4(secret_V1)))
#         secret_D = self.lrelu(self.bn72(self.deconv4(secret_D1)))

#         secret_C = torch.stack([secret_H,secret_V,secret_D], dim=2)
#         secret_image = self.ifm((secret_A,[secret_C]))
#         # if coeff:
#         #     return (secret_A,[secret_C])
#         # else:
#         return secret_image

In [112]:
class Decoder(nn.Module):
    def __init__(self,reveal_network):
        super().__init__()
        self.reveal_network = reveal_network
    
    def forward(self,hidden_image):     
        return self.reveal_network(hidden_image)

In [113]:

class SteganoModel(nn.Module):
    def __init__(self,encoder,decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self,cover_image,secret_image,hidden_image,mode):
        if mode == 'full':
            for param in self.encoder.parameters():
                param.requires_grad = True
            for param in self.decoder.parameters():
                param.requires_grad = False
            
            hidden_image = self.encoder(cover_image,secret_image)
            reveal_image = self.decoder(hidden_image)
            return hidden_image,reveal_image
        elif mode == 'encoder':
            for param in self.encoder.parameters():
                param.requires_grad = False
            for param in self.decoder.parameters():
                param.requires_grad = False

            return self.encoder(cover_image,secret_image)
        elif mode == 'decoder':
            for param in self.encoder.parameters():
                param.requires_grad = False
            for param in self.decoder.parameters():
                param.requires_grad = True
            return self.decoder(hidden_image)

In [114]:

# filename = os.path.join("/content/drive/MyDrive/Colab_Notebooks/Steganography/Completed_model2.joblib")
# # loaded_model = joblib.load(filename)
# joblib.dump(model, filename)

In [115]:
xfm = DWTForward(J=1, mode='zero', wave='db1').cuda()  # Accepts all wave types available to PyWavelets
ifm = DWTInverse(mode='zero', wave='db1').cuda()
prep_net = PrepNetwork(xfm)
hiding_network = HidingNetwork()
encoder = Encoder(prep_net,hiding_network)
reveal_net = RevealNetwork(ifm)
decoder = Decoder(reveal_net)
model = SteganoModel(encoder,decoder)
model.to(device)

SteganoModel(
  (encoder): Encoder(
    (prep_network): PrepNetwork(
      (xfm): DWTForward()
      (conv1): Conv2d(3, 6, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(6, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (hiding_network): HidingNetwork(
      (conv1): Conv2d(3, 6, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (conv2): Conv2d(6, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (bn12): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(18, 36, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (bn36): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv4): Conv2d(72, 144, kernel_size=(4, 4), stride=(2, 2), pad

In [116]:
class SteganoLoss(nn.Module):
    def __init__(self,cover_weight,secret_weight):
        super().__init__()
        self.cover_weight = cover_weight
        self.secret_weight = secret_weight
        # self.xfm = xfm
    def forward(self,predicted_cover_image,cover_image,
               predicted_secret_image,secret_image):
        # (secretcA,[secretcD]) = self.xfm(secret_image)
        # (psecretcA,[psecretcD]) = predicted_secret_image
        cover_loss = self.cover_weight*(F.mse_loss(predicted_cover_image,cover_image))
        secret_loss = self.secret_weight*(F.mse_loss(predicted_secret_image,secret_image)) 
        return cover_loss + secret_loss

class DecoderLoss(nn.Module):
    def __init__(self,decoder_loss_weight):
        super().__init__()
        self.decoder_loss_weight = decoder_loss_weight
        # self.xfm = xfm
    
    def forward(self,reveal_output,secret_image):
        # (secretcA,[secretcD]) = self.xfm(secret_image)
        # (psecretcA,[psecretcD]) = reveal_output
        return self.decoder_loss_weight*(F.mse_loss(reveal_output,secret_image)) 

In [117]:

# # training_csv_path = os.path.join(dataset_path,train_dataset.csv)
# # test_csv_path = os.path.join(dataset_path,test_csv)

training_dataset = SteganoDataset('/content/drive/MyDrive/Colab Notebooks/Steganography/MyData.csv',imagetransformation['train_transforms'],'train','complete')
# test_dataset = SteganoDataset(training_csv_path,imagetransformation['train_transforms'],'Train','complete')


In [118]:

# test_dataset = SteganoDataset(dataframe,imagetransformation['test_transforms'],'Valid')


train_data_loader = torch.utils.data.DataLoader(training_dataset, 
                                                batch_size = TRAIN_BATCH_SIZE, 
                                                shuffle=True,
                                               drop_last = True,
                                               num_workers = 0
                                               )
# valid_data_loader = torch.utils.data.DataLoader(valid_dataset, 
#                                                 batch_size = VALID_BATCH_SIZE, 
#                                                 shuffle=True,
#                                                 drop_last = True,
#                                                 num_workers = 0
#                                                )
                                               
# test_data_loader = torch.utils.data.DataLoader(test_dataset, 
#                                                 batch_size = VALID_BATCH_SIZE, 
#                                                 shuffle=True,
#                                                 drop_last = True,
#                                                 num_workers = 0
#                                                )



In [119]:

full_model_criterion = SteganoLoss(SECRET_LOSS_WEIGHT,COVER_LOSS_WEIGHT)

full_model_optimizer = torch.optim.Adam(model.parameters(),lr=LEARNING_RATE)

decoder_criterion = DecoderLoss(DECODER_LOSS_WEIGHT)
training_full_model_loss_list = []
decoder_loss_list = []


In [120]:

def train(model,epochs,decoder_criterion,full_model_optimizer,full_model_criterion,learning_rate,training_iterator):
    print_every=1
    counter=1

    for epoch in range(epochs):
        for index,training_dict in enumerate(training_iterator):
            cover_image = training_dict['cover_image']
            cover_image = cover_image.to(device)
            
            secret_image = training_dict['secret_image']
            secret_image = secret_image.to(device)
            counter=counter+1


            full_model_optimizer.zero_grad()

            encoder_output = model(cover_image,secret_image,secret_image,'encoder')

            hidden_image,reveal_image = model(cover_image,secret_image,secret_image,'full')
            
            full_model_loss = full_model_criterion(hidden_image,cover_image, reveal_image,secret_image)
            
            full_model_loss.backward()
            full_model_optimizer.step()
            
            full_model_optimizer.zero_grad()
            reveal_output = model(cover_image,secret_image,encoder_output,'decoder')
                                                                 
            decoder_loss = decoder_criterion(reveal_output,secret_image)

            decoder_loss.backward()
            full_model_optimizer.step()
            if counter % 50 == 0:
              print(full_model_loss)
            

        training_full_model_loss_list.append(full_model_loss)
        decoder_loss_list.append(decoder_loss)
        if epoch % print_every == 0:
            print("Training full model loss at {} epochs is: {}".format(epoch, full_model_loss))
            print("Training decoder loss at {} epochs is: {}".format(epoch, decoder_loss))

    return model, training_full_model_loss_list,decoder_loss_list


Training full model loss at 0 epochs is: 0.14862200617790222
Training decoder loss at 0 epochs is: 0.056159209460020065
Training full model loss at 50 epochs is: 0.010316253639757633
Training decoder loss at 50 epochs is: 0.006905762478709221
Training full model loss at 100 epochs is: 0.005915718153119087
Training decoder loss at 100 epochs is: 0.004135390743613243
Training full model loss at 150 epochs is: 0.00597141869366169
Training decoder loss at 150 epochs is: 0.003256219904869795
Training full model loss at 200 epochs is: 0.004562869668006897
Training decoder loss at 200 epochs is: 0.0032390698324888945
Training full model loss at 250 epochs is: 0.004033954814076424
Training decoder loss at 250 epochs is: 0.00324784847907722
Training full model loss at 300 epochs is: 0.0028372027445584536
Training decoder loss at 300 epochs is: 0.002142705488950014
Training full model loss at 350 epochs is: 0.003511040238663554
Training decoder loss at 350 epochs is: 0.0028315880335867405
Training full model loss at 400 epochs is: 0.006889783777296543
Training decoder loss at 400 epochs is: 0.006818610243499279
Training full model loss at 450 epochs is: 0.0030810432508587837
Training decoder loss at 450 epochs is: 0.0027422530110925436
Training full model loss at 500 epochs is: 0.0026186055038124323
Training decoder loss at 500 epochs is: 0.00222468632273376
Training full model loss at 550 epochs is: 0.0022935508750379086
Training decoder loss at 550 epochs is: 0.0016986110713332891
Training full model loss at 600 epochs is: 0.0017199815483763814
Training decoder loss at 600 epochs is: 0.0012137810699641705
Training full model loss at 650 epochs is: 0.002981864381581545
Training decoder loss at 650 epochs is: 0.002139737131074071
Training full model loss at 700 epochs is: 0.00157183688133955
Training decoder loss at 700 epochs is: 0.0011825517285615206
Training full model loss at 750 epochs is: 0.001144192647188902
Training decoder loss at 750 epochs is: 0.0006921886233612895
Training full model loss at 800 epochs is: 0.0016071755671873689
Training decoder loss at 800 epochs is: 0.0011148437624797225
Training full model loss at 850 epochs is: 0.0016389719676226377
Training decoder loss at 850 epochs is: 0.0010738647542893887
Training full model loss at 900 epochs is: 0.0011910195462405682
Training decoder loss at 900 epochs is: 0.0007727841730229557
Training full model loss at 950 epochs is: 0.0011191064259037375
Training decoder loss at 950 epochs is: 0.0007212755153886974

In [121]:
import os
import random
import shutil
import pandas as pd

# Set the path to the destination folders
secret_path = '/content/drive/MyDrive/Colab Notebooks/Dataset256/secret'
cover_path = '/content/drive/MyDrive/Colab Notebooks/Dataset256/cover'

# Get a list of all the images in the folder
secret_img_list = os.listdir(secret_path)
cover_img_list = os.listdir(cover_path)
print(len(secret_img_list))
print(len(cover_img_list))
# # Create a dataframe from the cover and secret image lists
# dataset = pd.DataFrame(columns=['cover_image', 'secret_image'])
# for i in range(len(cover_img_list)):
#     dataset = dataset.append({
#         'cover_image': cover_img_list[i],
#         'secret_image': secret_img_list[i]
#     }, ignore_index=True)

# # Save the dataframe as a CSV file
# dataset.to_csv('Dataset_x256.csv', index=False)


1000
991


In [122]:
model, training_full_model_loss_list, decoder_loss_list = train(model, EPOCHS,decoder_criterion, full_model_optimizer, full_model_criterion, LEARNING_RATE, train_data_loader)

1
2
3
4
5
6
7
torch.Size([16, 3, 3, 125, 125])
torch.Size([16, 3, 127, 127])


RuntimeError: ignored

In [ ]:
filename = os.path.join("/content/drive/MyDrive/Colab Notebooks/Steganography/unetmodel.pkl")
# loaded_model = joblib.load(filename)
torch.save(model,filename)

In [ ]:
# model = torch.load('/content/drive/MyDrive/Colab Notebooks/Steganography/stegomodel_latest.pkl')

In [ ]:

def predict(model,iterator,mode):

    predict_dict = next(iter(iterator))
    cover_image = predict_dict['cover_image']
    # print(cover_image)
    cover_image = cover_image.to(device)
    secret_image_1 = predict_dict['secret_image']
    secret_image_1 = secret_image_1.to(device)

    model.eval()

 
    if mode =='decoder':
      
      reveal_image_1= model(cover_image,cover_image,cover_image,mode)
      
      # dot_graph = torchviz.make_dot(model(cover_image,cover_image,cover_image,mode))
      # dot_graph.render("decoder.dot")
    elif mode =='encoder':
      hidden_image= model(cover_image,secret_image_1,secret_image_1,mode)
      # dot_graph = torchviz.make_dot(model(cover_image,secret_image_1,secret_image_1,mode))
      # dot_graph.render("encoder.dot")
    elif mode == "full":
      hidden_image,reveal_image_1= model(cover_image,secret_image_1,secret_image_1,mode)
      # dot_graph = torchviz.make_dot(model(cover_image,secret_image_1,secret_image_1,mode))
      # dot_graph.render("full.dot")
      cover_image = cover_image * 255
      cover_image = cover_image.to(torch.device('cpu'))
      cover_image = cover_image.detach().to(torch.long)
      secret_image_1 = secret_image_1 * 255
      secret_image_1 = secret_image_1.to(torch.device('cpu'))
      secret_image_1 = secret_image_1.detach().to(torch.long)
    if mode =='encoder' or mode == 'full':
      hidden_image[hidden_image>1] = 1
      hidden_image = hidden_image * 255
      hidden_image = hidden_image.to(torch.device('cpu'))
      hidden_image = hidden_image.detach().to(torch.long)
      h = hidden_image[0].permute(1,2,0).numpy()
      im = Image.fromarray(h.astype(np.uint8))
      im.save("/content/drive/MyDrive/Colab Notebooks/stego_img.png")
      if mode == 'encoder': return h
    if mode =='decoder' or mode == 'full':
      reveal_image_1[reveal_image_1>1] = 1
      reveal_image_1 = reveal_image_1 * 255
      reveal_image_1 = reveal_image_1.to(torch.device('cpu'))
      reveal_image_1 = reveal_image_1.detach().to(torch.long)
      r = reveal_image_1[0].permute(1,2,0).numpy()
      im = Image.fromarray(r.astype(np.uint8))
      im.save("/content/drive/MyDrive/Colab Notebooks/revealed.png")
      if mode == 'encoder': return r
    return {
        'cover_image_grid':cover_image[0].permute(1,2,0).numpy(),
        'secret_image_1_grid':secret_image_1[0].permute(1,2,0).numpy(),
        'hidden_image_grid':h,
        'reveal_image_1_grid':r,
    }
grids = predict(model,train_data_loader,'full')

In [ ]:
import torch
import math, time, sys
from PIL import Image

class Arnold:

    def __init__(self, a:int, b:int, rounds:int):
        # Parameters
        self.a = a
        self.b = b
        self.rounds = rounds

    def mapping(self, s:torch.Size):
        x, y = torch.meshgrid(torch.arange(s[0]), torch.arange(s[1]))
        xmap = (self.a*self.b*x + x + self.a*y) % s[0]
        ymap = (self.b*x + y) % s[1]
        return xmap, ymap

    def inverseMapping(self, s:torch.Size):
        x, y = torch.meshgrid(torch.arange(s[0]), torch.arange(s[1]))
        xmap = (x - self.a*y) % s[0]
        ymap = (-self.b*x + self.a*self.b*y + y) % s[1]
        return xmap, ymap

    def applyTransformTo(self, image:torch.Tensor):
        xm, ym = self.mapping(image.shape[-2:])
        img = image
        for r in range(self.rounds):
            img = img[..., xm, ym]
        return img

    def applyInverseTransformTo(self, image:torch.Tensor):
        xm, ym = self.inverseMapping(image.shape[-2:])
        img = image
        for r in range(self.rounds):
            img = img[..., xm, ym]
        return img


In [ ]:
# grids['cover_image_grid'] = arnold.applyInverseTransformTo(grids['cover_image_grid'])
# grids['hidden_image_grid'] = arnold.applyInverseTransformTo(grids['hidden_image_grid'])

In [ ]:
def plot(grids):
    plt.figure(figsize=(15,8))
    
    plt.subplot(241)
    plt.title('Cover Image')
    plt.imshow(grids['cover_image_grid'])

    plt.subplot(242)
    plt.title('Secret Image')
    plt.imshow(grids['secret_image_1_grid'])

    plt.subplot(245)
    plt.title('Hidden Image')
    plt.imshow(grids['hidden_image_grid'])
    
    plt.subplot(246)
    plt.title('Reveal Image')
    plt.imshow(grids['reveal_image_1_grid'])

    plt.savefig('/content/drive/MyDrive/Colab Notebooks/Plot.png')
    plt.show()
    

In [ ]:
plot(grids)

In [ ]:
import pickle
import torchviz


In [ ]:
!pip install torchviz

In [ ]:
!apt-get install -y graphviz


In [ ]:
!dot -Tpng full.dot -o full.png

In [ ]:
!dot -Tpng encoder.dot -o encodermodel.png

In [ ]:
!dot -Tpng decoder.dot -o decodermodel.png

In [ ]:
torchviz.render(model, "model.png")

In [ ]:
# def predict(cover,model=test):
#   mode='decoder'
#   cover=torchvision.transforms.Compose([torchvision.transforms.Resize((512,512)),torchvision.transforms.ToTensor(cover)])
#   cover = cover.to(device)
#   model.eval()
#   reveal_image_1= model(cover,cover,cover,mode)
#   reveal_image_1[reveal_image_1>1] = 1
#   reveal_image_1 = reveal_image_1 * 255
#   reveal_image_1 = reveal_image_1.to(torch.device('cpu'))
#   reveal_image_1 = reveal_image_1.detach().to(torch.long)
#   r = reveal_image_1[0].permute(1,2,0).numpy()
#   im = Image.fromarray(r.astype(np.uint8))
#   return im

In [ ]:
from torchvision.utils import save_image


In [ ]:
'''
data = next(iter(test_data_loader))

cover_image = data['cover_image']
cover_image = cover_image.to(device)

secret_image_1 = data['secret_image']
secret_image_1 = secret_image_1.to(device)

model.eval()
    
hidden_image,reveal_image_1 = model(cover_image,secret_image_1,secret_image_1,'full')
'''

In [ ]:
#make_dot(hidden_image).render('hidden_image',format='png')

In [ ]:
#make_dot(reveal_image_1).render('reveal_image',format='png')

In [ ]:

def plot_loss(loss_list,num):
    plt.plot(torch.tensor(loss_list, device = 'cpu'))
    if(num==1):
      plt.title('full mode loss')
      plt.xlabel("epochs")
      plt.ylabel("loss")
      plt.savefig('/content/drive/MyDrive/Colab Notebooks/fullmodeloss11.png')
      plt.show()
    else:
      plt.title('decoder loss')
      plt.xlabel("epochs")
      plt.ylabel("loss")
      plt.savefig('/content/drive/MyDrive/Colab Notebooks/decoderloss11.png')
      plt.show()


In [ ]:
plot_loss(training_full_model_loss_list,1)

In [ ]:
plot_loss(decoder_loss_list,2)

In [ ]:

for i in range(5):
    grids = predict(test,test_data_loader,'full')
    plot(grids)


In [ ]:
def wtransform(img):
  image = np.array(img)
  coeffs = [[],[],[]]

  for ch in range(3):
    # Perform wavelet transform
    coeffs[ch] = pywt.swt2(image[:,:,ch], 'haar',level=1)
  return coeffs, image
def iwtransform(coeffs,image):
  image[:,:,0] = pywt.iswt2(coeffs[0], 'haar')
  image[:,:,1] = pywt.iswt2(coeffs[1], 'haar')
  image[:,:,2] = pywt.iswt2(coeffs[2], 'haar')
  return Image.fromarray(np.uint8(image))

In [ ]:
from matplotlib.image import imread
import matplotlib.pyplot as plt
import pywt
plt.rcParams['figure.figsize'] = [16, 16]
plt.rcParams.update({'font.size': 18})

A = Image.open('/content/drive/MyDrive/Colab Notebooks/Steganography/NikeshC.jpg')
print(type(A))
B = np.mean(A, -1); # Convert RGB to grayscale

n = 1
w = 'db1'
coeff = [[],[]]
arr = [[],[],[]]
coeff_slices = [[],[],[]]
for ch in range(3):
  coeffs = pywt.swt2(A[:,:,ch],wavelet=w,level=n)
  # data = np.zeros_like(A)
  # print(data.shape)
  # print(coeffs[0])
  # coeffs[1] = pywt.wavedec2(A[:,:,1],wavelet=w,level=n)
  # coeffs[2] = pywt.wavedec2(A[:,:,2],wavelet=w,level=n)

  # normalize each coefficient array
  
  coeff[0] = coeffs[0][0] / np.abs(coeffs[0][0]).max()
  for detail_level in range(n):
      coeff[detail_level + 1] = [d/np.abs(d).max() for d in coeffs[0][detail_level + 1]]

  arr[ch], coeff_slices[ch] = np.array(pywt.coeffs_to_array(coeff))

final_img = np.dstack([arr[0]*255, arr[1]*255, arr[2]*255]).astype(np.uint8)
print(final_img.shape)

# a = arr[coeff_slices[1]['dd']]
plt.imshow(final_img,cmap='gray',vmin=-0.25,vmax=0.75)
plt.savefig('/content/drive/MyDrive/Colab Notebooks/Plotwave4.png')
plt.show()

In [ ]:

r_np = imread('/content/drive/MyDrive/Colab Notebooks/Plotwave2.png')
g_np = imread('/content/drive/MyDrive/Colab Notebooks/Plotwave3.png')
b_np = imread('/content/drive/MyDrive/Colab Notebooks/Plotwave4.png')


# Add the channels to the final image
final_img = np.dstack([b_np[:,:,0], g_np[:,:,2], r_np[:,:,2]]).astype(np.uint8)

print(final_img.shape)
plt.imshow(final_img, interpolation='nearest')
plt.show()

In [ ]:
import numpy as np
import math, time, sys
from PIL import Image
class Arnold:

    def __init__(self, a:int, b:int, rounds:int):
        # Parameters
        self.a = a
        self.b = b
        self.rounds = rounds

    def mapping(self, s:np.shape):
        x, y = np.meshgrid(range(s[0]), range(s[0]), indexing="ij")
        xmap = (self.a*self.b*x + x + self.a*y) % s[0]
        ymap = (self.b*x + y) % s[0]
        return xmap, ymap

    def inverseMapping(self, s:np.shape):
        x, y = np.meshgrid(range(s[0]), range(s[0]), indexing="ij")
        xmap = (x - self.a*y) % s[0]
        ymap = (-self.b*x + self.a*self.b*y + y) % s[0]
        return xmap, ymap

    def applyTransformTo(self, image:np.ndarray):
        xm, ym = self.mapping(image.shape)
        img = image
        for r in range(self.rounds):
            img = img[xm, ym]
        return img

    def applyInverseTransformTo(self, image:np.ndarray):
        xm, ym = self.inverseMapping(image.shape)
        img = image
        for r in range(self.rounds):
          img = img[xm, ym]
        return img

In [ ]:

image_path = "/content/drive/MyDrive/Colab Notebooks/Steganography/Nikesh.jpg"

# Arnold Transform Parameters
a = 6
b = 40
rounds = 33

# Open the images
lena = np.array(Image.open(image_path))

print(" ~~~~  * PARAMETERS *  ~~~~ ")
arnold = Arnold(a, b, rounds)
print("\ta:\t", a)
print("\tb:\t", b)
print("\trounds:\t", rounds)

print("\n ~~~~  *  RESULTS   *  ~~~~ ")

start_time = time.time()
scrambled = arnold.applyTransformTo(lena)
exec_time = time.time() - start_time
print("Transform  execution time: %.6f " % exec_time, "sec")
im = Image.fromarray(scrambled)
im.save("/content/drive/MyDrive/Colab Notebooks/Steganography/scrambledInvert.png", format="PNG")

start_time = time.time()
reconstructed = arnold.applyInverseTransformTo(scrambled)
exec_time = time.time() - start_time
print("Inverse T. execution time: %.6f " % exec_time, "sec")
im = Image.fromarray(reconstructed)
im.save("/content/drive/MyDrive/Colab Notebooks/reconstructed", format="PNG")


<h1>Analysis Code</h1>

In [ ]:
import math
import numpy as np
import cv2
from matplotlib.image import imread
from scipy.stats import entropy
import matplotlib.pyplot as plt

def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def snr(stego_image, original_image):
    # Calculate the noise image
    noise_image = stego_image - original_image
    # Calculate the average noise power
    noise_power = np.mean(noise_image ** 2)
    # Calculate the average signal power
    signal_power = np.mean(original_image ** 2)
    # Calculate the SNR
    snr = signal_power / noise_power
    # Convert the SNR to decibels
    snr_db = 10 * np.log10(snr)
    return snr_db

def ssim(image1, image2):
    # Convert the images to grayscale
    image1_gray = cv2.cvtColor(cv2.convertScaleAbs(image1), cv2.COLOR_BGR2GRAY)
    image2_gray = cv2.cvtColor(cv2.convertScaleAbs(image2), cv2.COLOR_BGR2GRAY)
    # Calculate the SSIM
    ssim, _ = cv2.quality.QualityMSE_compute(image1_gray, image2_gray)
    return ssim

def correlation_coefficient(image1, image2):
    # Flatten the images into 1D arrays
    image1_flat = image1.flatten()
    image2_flat = image2.flatten()
    # Calculate the correlation coefficient
    correlation = np.corrcoef(image1_flat, image2_flat)[0, 1]
    return correlation

def image_entropy(image):
    # Flatten the image into a 1D array
    image_flat = image.flatten()
    # Calculate the entropy
    return entropy(image_flat)

def pixel_frequencies(image):
    # Flatten the image into a 1D array
    image_flat = image.flatten()
    # Calculate the frequencies of each pixel value
    unique, counts = np.unique(image_flat, return_counts=True)
    frequencies = counts / len(image_flat)
    return unique, frequencies,counts

def plot_frequencies(unique, frequencies,counts):
    # Plot the frequencies
    plt.plot(unique, counts)
    plt.xlim([0, 256])
    plt.show()

<h1>For Loop</h1>

In [ ]:
cover = Image.open(cover)
secret = Image.open(secret)
grids = predict(model,test_data_loader,'full')

In [ ]:
cpath = '/content/drive/MyDrive/Colab Notebooks/Steganography/Dataset/cover'
spath = '/content/drive/MyDrive/Colab Notebooks/Steganography/Dataset/train'
listofcimg = os.listdir(cpath)
listofsimg = os.listdir(spath)

In [ ]:
df = pd.DataFrame(columns=["psnrC", "psnrS", "snrC", "snrS", "ssimC", "ssimS", "corrC", "corrS", "entC", "entH", "entS", "entR"])
for i in range(1000):
  dataset = [{
        'cover_image':os.path.join(cpath,listofcimg[i]),
        'secret_image':os.path.join(spath,listofsimg[i])
    }]
  dataframe = pd.DataFrame(dataset)
  test_dataset = SteganoDataset(dataframe,imagetransformation['test_transforms'],'Valid')                                               
  test_data_loader = torch.utils.data.DataLoader(test_dataset, 
                                                  batch_size = VALID_BATCH_SIZE, 
                                                  shuffle=True,
                                                  drop_last = True,
                                                  num_workers = 0
                                                )

  grids = predict(model,test_data_loader,'full')
  psnrC = psnr(grids['cover_image_grid'],grids['hidden_image_grid'])
  psnrS = psnr(grids['secret_image_1_grid'],grids['reveal_image_1_grid'])
  snrC = snr(grids['cover_image_grid'],grids['hidden_image_grid'])
  snrS = snr(grids['secret_image_1_grid'],grids['reveal_image_1_grid'])
  ssimC = ssim(grids['cover_image_grid'],grids['hidden_image_grid'])[0]
  ssimS = ssim(grids['secret_image_1_grid'],grids['reveal_image_1_grid'])[0]
  corrC = correlation_coefficient(grids['cover_image_grid'],grids['hidden_image_grid'])
  corrS = correlation_coefficient(grids['secret_image_1_grid'],grids['reveal_image_1_grid'])
  entC = image_entropy(grids['cover_image_grid'])
  entH = image_entropy(grids['hidden_image_grid'])
  entS = image_entropy(grids['secret_image_1_grid'])
  entR = image_entropy(grids['reveal_image_1_grid'])
  df = df.append({"psnrC": psnrC, "psnrS": psnrS, "snrC": snrC, "snrS": snrS, "ssimC": ssimC, "ssimS": ssimS, "corrC": corrC, "corrS": corrS, "entC": entC, "entH": entH, "entS": entS, "entR": entR}, ignore_index=True)

df.to_csv("/content/drive/MyDrive/Colab Notebooks/Steganography/table.csv", index=False)

In [ ]:
df_norm = df.apply(lambda x: (x - x.mean()) / x.std())

In [ ]:
df.plot(y='psnrS')
plt.xlabel("Image number")
plt.ylabel("PSNR")
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(df.corr())

In [ ]:
df.mean(axis=0)

In [ ]:
from skimage.metrics import structural_similarity as ssim

In [ ]:
ssimValue = ssim(grids['cover_image_grid'],grids['secret_image_1_grid'],win_size=11, multichannel = True)

In [ ]:
ssimValue

In [ ]:
from pyssim import SSIM 
ssim = SSIM()
score = ssim.compute(grids['cover_image_grid'],grids['secret_image_1_grid'])

In [ ]:
!pip install pyssim